# JUICE RPWI HF SID2 (RAW): L1a CAL  for ASW1/2/3 -- 2025/7/10

# Import lib

In [ ]:
import copy
import csv
import datetime
import math
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import sys
from scipy.signal import find_peaks   # added

# Library setting (same with "sample")

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf
import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_1-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cal_lib   as juice_cal
import juice_cdf_lib   as juice_cdf
import juice_sid2_data as juice_data
import juice_sid2_lib  as juice_sid2
import juice_spec_lib  as juice_spec

# Mode seting (same with "sample")

In [ ]:
# *** Plot dump ***
dump_mode = 0                           # 0: no-dump  1:plot dump  2:comparison for operation
# *** CAL ***
unit_mode = 0                           # [Power]     0: raw     1: V＠ADC     2: V@HF    3: V@RWI  4: V/m
band_mode = 0                           # [Power]     0: sum     1: /Hz
cal_mode  = 2                           # [Power]     0: background     1: CAL     2: all
# *** Unit ***
f_mode    = 1                           # [Frequency] 0: linear  1: log
time_mode = 1                           # [Time]      0: Epoch   1: data number
gap_mode  = 0                           # [gap]       0: n/a     1: gap
comp_mode = 1                           # [Complex]   0: Poweer  1: Matrix   3: Matrix-2D  >3: any
# *** Frenquency in Linear ***
f_mode_min = 80;  f_mode_max = 2068     # 80 - 2068 : as same as SID-4/20
# *** Power range ***
p_raw_max = 7.0                         # background: 7.5   CAL: 10
p_raw_min = -1.0                        # background: 2.5   CAL: 5
# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder

In [ ]:
E_min = '2024-08-19 21:05:00';  t_min0 = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S")
E_max = '2024-08-19 21:10:00';  t_max0 = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S")
t_min0 = 0;  t_max0 = 0

#### (special)

In [ ]:
# *** CAL make modes ***
CAL_make_mode  = 0                      # 0: n/a      1: make

# get CDF data (same with "sample")

In [ ]:
# date='20230419';  ver = 'V03'
# date='20230530';  ver = 'V03'   # CAL
# date='20230601';  ver = 'V03'
# date='20230712';  ver = 'V03'
# date='20230713';  ver = 'V03'
# date='20240125';  ver = 'V02'
# date='20240822';  ver = 'V02'   # CAL
# date='20250331';  ver = 'V01'   # CAL
date='0';         ver = 'V02'   # Test data
data_dir, data_name_list = juice_data.datalist(date, ver)       # [date]   yyyymmdd: group read    others: file list

In [ ]:
class struct:
    pass

data = struct()
num_list = len(data_name_list)
for i in range(num_list):
    data_name = data_name_list[i];  cdf_file = data_dir + data_name;  print(i, cdf_file)
    RPWI_FSW_version = juice_cdf._RPWI_FSW_version(cdf_file)
    cdf = pycdf.CDF(cdf_file);      data1 = juice_sid2.hf_sid2_read(cdf, RPWI_FSW_version)
    if i==0:
        data = data1;                                print(data.Eu_i.shape)
    else:
        data = juice_sid2.hf_sid2_add(data, data1);  print(data.Eu_i.shape)
data_name = os.path.split(data_name)[1];             print("data name:", data_name)

In [ ]:
data = juice_sid2.hf_sid2_shaping(data, cal_mode)

In [ ]:
date1 = data.epoch[0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]; date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2
f_min0  = data.frequency[0][0][0]; f_max0  = max(np.ravel(data.frequency))

print("       date and time:", str_date)
print("         Num-samples:", data.n_time, "  Num-Frequency:", data.n_step, "  Length:", data.n_samp)
print("         Time-length:", data.time[0][0][-1], "sec in 1-sweep, with time resolution of ", data.time[0][0][1], "sec")
print("f, width, step (kHz):", f_min0,  "-", f_max0, data.freq_width[0][0][0], data.freq_step[0][0][0])
print("                  Ch:", data.ch_selected[data.n_time//2])

# Wave data (same with "sample")

In [ ]:
T_HF  = data.T_HF_FPGA[data.n_time//2];  T_RWI = (data.T_RWI_CH1[data.n_time//2] + data.T_RWI_CH2[data.n_time//2])/2
if T_HF  > 199 or T_HF  < -50  or math.isnan(T_HF):
   T_HF  = 25;  data.T_HF_FPGA[:] = math.nan
if T_RWI > 199 or T_RWI < -199 or math.isnan(T_RWI):
   T_RWI = 25;  data.T_RWI_CH1[:] = math.nan;  data.T_RWI_CH2[:] = math.nan

print("Temperature: ", 'HF - {:.1f}'.format(T_HF), '  RWI1 - {:.1f}'.format(T_RWI))

In [ ]:
wave = juice_cal.wave_cal(data, 2, unit_mode, T_HF, T_RWI)
p_max0 = p_raw_max + wave.cf/10;     p_min0 = p_raw_min + wave.cf/10
p_max0 = np.ceil(np.log10( np.nanmax( [np.nanmax(wave.Eu_i), np.nanmax(wave.Ev_i), np.nanmax(wave.Ew_i), np.nanmax(wave.Eu_q), np.nanmax(wave.Ev_q), np.nanmax(wave.Ew_q)] ) )*5)/5+.5

print("unit_mode:", unit_mode, "  conversion factor:", '{:.1f}'.format(wave.cf), 
      "  MAX-min:", '{:.1f}'.format(p_max0), '{:.1f}'.format(p_min0), "  str_unit:", wave.str_unit, 
      "  T-HF & T-RWI:", T_HF, "(", data.T_HF_FPGA[data.n_time//2], ")", T_RWI, "(", data.T_RWI_CH1[data.n_time//2], ")")

In [ ]:
print("decimation:", juice_cdf._sample_rate(data.decimation[0]))
print("Peak Eu:", '{:.2e}'.format(np.nanmax(wave.Eu_i)), ' {:.2e}'.format(np.nanmax(wave.Eu_q)), wave.str_unit)
print("Peak Ev:", '{:.2e}'.format(np.nanmax(wave.Ev_i)), ' {:.2e}'.format(np.nanmax(wave.Ev_q)), wave.str_unit)
print("Peak Ew:", '{:.2e}'.format(np.nanmax(wave.Ew_i)), ' {:.2e}'.format(np.nanmax(wave.Ew_q)), wave.str_unit)
if unit_mode==0:
    print("Peak Eu:", '{:x}'.format(np.int64(np.nanmax(wave.Eu_i))), ' {:x}'.format(np.int64(np.nanmax(wave.Eu_q))), wave.str_unit)
    print("Peak Ev:", '{:x}'.format(np.int64(np.nanmax(wave.Ev_i))), ' {:x}'.format(np.int64(np.nanmax(wave.Ev_q))), wave.str_unit)
    print("Peak Ew:", '{:x}'.format(np.int64(np.nanmax(wave.Ew_i))), ' {:x}'.format(np.int64(np.nanmax(wave.Ew_q))), wave.str_unit)

In [ ]:
fig = plt.figure(figsize=(16, 11))
ax1 = fig.add_subplot(6, 1, 1);  ax2 = fig.add_subplot(6, 1, 2);  ax3 = fig.add_subplot(6, 1, 3)
ax4 = fig.add_subplot(6, 1, 4);  ax5 = fig.add_subplot(6, 1, 5);  ax6 = fig.add_subplot(6, 1, 6)

ax1.plot(np.ravel(wave.Eu_i[:][:]), '-r', linewidth=.5, label='Eu_i'); ax1.plot(np.ravel(wave.Eu_q[:][:]), ':g', linewidth=.5, label='Eu_q')
ax2.plot(np.ravel(wave.Ev_i[:][:]), '-r', linewidth=.5, label='Ev_i'); ax2.plot(np.ravel(wave.Ev_q[:][:]), ':g', linewidth=.5, label='Ev_q')
ax3.plot(np.ravel(wave.Ew_i[:][:]), '-r', linewidth=.5, label='Ew_i'); ax3.plot(np.ravel(wave.Ew_q[:][:]), ':g', linewidth=.5, label='Ew_q')
ax4.plot(np.ravel(data.frequency),     '-b', linewidth=.5,  label='Frequency')
ax4.plot(np.ravel(data.freq_step*10),  '-g', linewidth=0.8, label='step*10')
ax4.plot(np.ravel(data.freq_width*10), ':b', linewidth=1.0, label='width*10')
ax5.plot(np.ravel(data.T_HF_FPGA),  ':r', label='T (HK-FPGA)');  ax5.plot(np.ravel(data.T_RWI_CH1),     ':b', label='T (RWI1)');  
ax5.plot(np.ravel(data.T_RWI_CH2),  ':g', label='T (RWI2)');     ax5.plot(np.ravel(data.cal_signal*10), '-k', label='CAL-Singal')
ax6.plot(np.ravel(data.epoch[:]), '.')

ax1.set_ylabel('Eu '+wave.str_unit);     ax2.set_ylabel('Ev '+wave.str_unit);  ax3.set_ylabel('Ew '+wave.str_unit);  
ax4.set_ylabel('Frequency [kHz]');  ax5.set_ylabel('T [degC]');      ax6.set_xlabel(str_date)
title_label = '[JUICE/RPWI HF RAW (SID-2)]\n' + data_name;           ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8)

xlim=[-0.5, len(np.ravel(data.Eu_i[:][:])) -0.5];  ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
xlim=[-0.5, data.n_time                        -0.5];  ax5.set_xlim(xlim);  ax6.set_xlim(xlim)
p_max = np.ceil(np.log10(np.nanmax([np.nanmax(wave.Eu_i), np.nanmax(wave.Eu_q), np.nanmax(wave.Ev_i), np.nanmax(wave.Ev_q), np.nanmax(wave.Ew_i), np.nanmax(wave.Ew_q)]))*5)/5
ylim=[-10**(p_max), 10**(p_max)];  ax1.set_ylim(ylim);  ax2.set_ylim(ylim);  ax3.set_ylim(ylim)
ylim=[f_min0, f_max0];             ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

### WAVE - First 3 Sweeps (same with "sample")

In [ ]:
num_sweep = 3;  n_sweep0 = data.n_time//2;  n_sweep1 = n_sweep0 + num_sweep;  print("[specific sweep]  SWEEP:", n_sweep0, n_sweep1, "in total of", data.n_time)
f_min = f_min0;  f_max = f_max0

fig = plt.figure(figsize=(16, 11))
ax1 = fig.add_subplot(5, 1, 1);  ax2 = fig.add_subplot(5, 1, 2);  ax3 = fig.add_subplot(5, 1, 3);  ax4 = fig.add_subplot(5, 1, 4);  ax5 = fig.add_subplot(5, 1, 5)

ax1.plot(np.ravel(wave.Eu_i[n_sweep0:n_sweep1]), '-r', linewidth=.5, label='Eu_i');  ax1.plot(np.ravel(wave.Eu_q[n_sweep0:n_sweep1]), ':g', linewidth=.5, label='Eu_q')
ax2.plot(np.ravel(wave.Ev_i[n_sweep0:n_sweep1]), '-r', linewidth=.5, label='Ev_i');  ax2.plot(np.ravel(wave.Ev_q[n_sweep0:n_sweep1]), ':g', linewidth=.5, label='Ev_q')
ax3.plot(np.ravel(wave.Ew_i[n_sweep0:n_sweep1]), '-r', linewidth=.5, label='Ew_i');  ax3.plot(np.ravel(wave.Ew_q[n_sweep0:n_sweep1]), ':g', linewidth=.5, label='Ew_q')
ax4.plot(np.ravel(data.frequency[n_sweep0:n_sweep1]),     '-b', linewidth=.5,  label='Frequency')
ax4.plot(np.ravel(data.freq_step[n_sweep0:n_sweep1]*10),  '-g', linewidth=0.8, label='step*10')
ax4.plot(np.ravel(data.freq_width[n_sweep0:n_sweep1]*10), ':b', linewidth=1.0, label='width*10')
ax5.plot(np.ravel(data.epoch[n_sweep0:n_sweep1+1]), '.')

date1 = data.epoch[n_sweep0];  date1 = date1.strftime('%Y/%m/%d %R:%S');  ax5.set_xlabel(date1)
ax1.set_ylabel('Eu '+wave.str_unit);  ax2.set_ylabel('Ev '+wave.str_unit);  ax3.set_ylabel('Ew '+wave.str_unit);  ax4.set_ylabel('Frequency [kHz]');  
title_label = '[JUICE/RPWI HF RAW (SID-2)]\n' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8)

xlim=[-0.5, len(np.ravel(data.Eu_i[n_sweep0:n_sweep1])) -0.5]; ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim)
xlim=[-0.5, num_sweep                                   -0.5]; ax5.set_xlim(xlim)
p_max = np.ceil(np.log10( np.nanmax( [np.nanmax(wave.Eu_i[n_sweep0:n_sweep1]), np.nanmax(wave.Eu_q[n_sweep0:n_sweep1]), np.nanmax(wave.Ev_i[n_sweep0:n_sweep1]), np.nanmax(wave.Ev_q[n_sweep0:n_sweep1]), np.nanmax(wave.Ew_i[n_sweep0:n_sweep1]), np.nanmax(wave.Ew_q[n_sweep0:n_sweep1])] ) )*5)/5
ylim=[-10**(p_max), 10**(p_max)]; ax1.set_ylim(ylim);  ax2.set_ylim(ylim);  ax3.set_ylim(ylim)
ylim=[f_min, f_max]; ax4.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw-sweep.png'
    fig.savefig(png_fname)

### WAVE - First 3 steps (same with "sample")

In [ ]:
num_step = 3;  n_sweep = data.n_time//2;  n_step0 = data.n_step//2;  n_step1 = n_step0 + num_step;  print("[specific sweep]  SWEEP:", n_sweep, "in total of", data.n_time,  "[Steps]", n_step0, "-", n_step1)
fig = plt.figure(figsize=(16, 11))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(np.ravel(wave.Eu_i[n_sweep][n_step0:n_step1]), '-r', linewidth=.5, label='Eu_i');  ax1.plot(np.ravel(wave.Eu_q[n_sweep][n_step0:n_step1]), ':g', linewidth=.5, label='Eu_q')
ax2.plot(np.ravel(wave.Ev_i[n_sweep][n_step0:n_step1]), '-r', linewidth=.5, label='Ev_i');  ax2.plot(np.ravel(wave.Ev_q[n_sweep][n_step0:n_step1]), ':g', linewidth=.5, label='Ev_q')
ax3.plot(np.ravel(wave.Ew_i[n_sweep][n_step0:n_step1]), '-r', linewidth=.5, label='Ew_i');  ax3.plot(np.ravel(wave.Ew_q[n_sweep][n_step0:n_step1]), ':g', linewidth=.5, label='Ew_q')
ax4.plot(np.ravel(data.frequency[n_sweep][n_step0:n_step1]),     '-b', linewidth=.5, label='Frequency')

date1 = data.epoch[n_sweep];  date1 = date1.strftime('%Y/%m/%d %R:%S');  ax4.set_xlabel(date1)
ax1.set_ylabel('Eu '+wave.str_unit);  ax2.set_ylabel('Ev '+wave.str_unit);  ax3.set_ylabel('Ew '+wave.str_unit);  ax4.set_ylabel('Frequency [kHz]');  
title_label = '[JUICE/RPWI HF RAW (SID-2)]\n' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8)

xlim=[-0.5, len(np.ravel(data.Eu_i[n_sweep][n_step0:n_step1])) -0.5];  ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
p_max = np.ceil(np.log10( np.nanmax( [np.nanmax(wave.Eu_i[n_sweep][n_step0:n_step1]), np.nanmax(wave.Eu_q[n_sweep][n_step0:n_step1]), np.nanmax(wave.Ev_i[n_sweep][n_step0:n_step1]), np.nanmax(wave.Ev_q[n_sweep][n_step0:n_step1]), np.nanmax(wave.Ew_i[n_sweep][n_step0:n_step1]), np.nanmax(wave.Ew_q[n_sweep][n_step0:n_step1])] ) )*5)/5
ylim=[-10**(p_max), 10**(p_max)]; ax1.set_ylim(ylim);  ax2.set_ylim(ylim);  ax3.set_ylim(ylim)

# Plot
fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw-step.png'
    fig.savefig(png_fname)

# Spectrum Data (same with "sample")

In [ ]:
spec = juice_spec.hf_getspec_sid2  (data)                               # Wave -> Spectrum
spec = juice_cal.spec_cal(spec, 2, unit_mode, band_mode, T_HF, T_RWI)   # CAL
spec.EE = spec.EuEu + spec.EvEv + spec.EwEw

spec.epoch = spec.epoch.tolist();         spec.n_time = spec.EuEu.shape[0];  
num_1d    = np.arange(spec.n_time)
freq_1d   = spec.freq  [spec.n_time//2];  n_freq1 = freq_1d.shape[0];    f_min1 = freq_1d[0];  f_max1  = freq_1d[-1]
freq_w_1d = spec.freq_w[spec.n_time//2]

In [ ]:
p_max1 = p_raw_max + spec.cf/10;  p_min1 = p_raw_min + spec.cf/10
p_max1 = np.ceil(np.log10( np.nanmax( [np.nanmax(spec.EuEu), np.nanmax(spec.EvEv), np.nanmax(spec.EwEw)] ) )*5)/5+.5
# p_min1 = np.ceil(np.log10( np.nanmin( [np.nanmin(spec.EuEu), np.nanmin(spec.EvEv), np.nanmin(spec.EwEw)] ) )*5)/5+.5

print(spec.EuEu.shape, "[frequency]", f_min1, "-", f_max1, "kHz  (df =", freq_w_1d[0], "-", freq_w_1d[-1], "kHz)")
print(spec.str_unit, "(unit_mode:", unit_mode, ")  conversion-factor:", '{:.1f}'.format(spec.cf), "[wave:", '{:.1f}'.format(wave.cf),
      "]  T-HF & T-RWI:", T_HF, "(", data.T_HF_FPGA[data.n_time//2], ")", T_RWI, "(", data.T_RWI_CH1[data.n_time//2], ")")
print("[MAX-min]", '{:.1f}'.format(p_max1), "(", '{:.1f}'.format(p_max0), ')  {:.1f}'.format(p_min1), "(", '{:.1f}'.format(p_min0), ")")

In [ ]:
print("Peak   Eu:", '{:.2e}'.format(np.nanmax(wave.Eu_i)), ' {:.2e}'.format(np.nanmax(wave.Eu_q)), wave.str_unit)
print("Peak   Ev:", '{:.2e}'.format(np.nanmax(wave.Ev_i)), ' {:.2e}'.format(np.nanmax(wave.Ev_q)), wave.str_unit)
print("Peak   Ew:", '{:.2e}'.format(np.nanmax(wave.Ew_i)), ' {:.2e}'.format(np.nanmax(wave.Ew_q)), wave.str_unit)
#
peak_E = np.ravel(spec.EuEu[:][:]); peak_f = np.nanargmax(peak_E)
print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str_unit, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5), wave.str_unit)
peak_E = np.ravel(spec.EvEv[:][:]); peak_f = np.nanargmax(peak_E)
print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str_unit, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5), wave.str_unit)
peak_E = np.ravel(spec.EwEw[:][:]); peak_f = np.nanargmax(peak_E)
print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str_unit, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5), wave.str_unit)
if unit_mode==0:
    peak_E = np.ravel(spec.EuEu[:][:]); peak_f = np.nanargmax(peak_E);  print("(RAW)EuEu:", '{:10.0f}'.format(peak_E[peak_f]), '{:x}'.format(np.int64(peak_E[peak_f])))
    peak_E = np.ravel(spec.EvEv[:][:]); peak_f = np.nanargmax(peak_E);  print("(RAW)EvEv:", '{:10.0f}'.format(peak_E[peak_f]), '{:x}'.format(np.int64(peak_E[peak_f])))
    peak_E = np.ravel(spec.EwEw[:][:]); peak_f = np.nanargmax(peak_E);  print("(RAW)EwEw:", '{:10.0f}'.format(peak_E[peak_f]), '{:x}'.format(np.int64(peak_E[peak_f])))

In [ ]:
n_sweep1 = 0;  n_sweep2 = spec.n_time//2;  n_sweep3 = spec.n_time-1
"""
while np.isnan(np.ravel(spec.EuEu[n_sweep1][0])):
    n_sweep1 += 1
while np.isnan(np.ravel(spec.EuEu[n_sweep2][0])):
    n_sweep2 += 1
while np.isnan(np.ravel(spec.EuEu[n_sweep3][0])):
    n_sweep3 -= 1
"""
print(f"[First peak - {n_sweep1}]")
peak_E = np.ravel(spec.EuEu[n_sweep1]); peak_f = np.nanargmax(peak_E); print("EuEu:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EvEv[n_sweep1]); peak_f = np.nanargmax(peak_E); print("EvEv:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EwEw[n_sweep1]); peak_f = np.nanargmax(peak_E); print("EwEw:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
print(f"[Mid   peak - {n_sweep2}]")
peak_E = np.ravel(spec.EuEu[n_sweep2]); peak_f = np.nanargmax(peak_E); print("EuEu:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EvEv[n_sweep2]); peak_f = np.nanargmax(peak_E); print("EvEv:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EwEw[n_sweep2]); peak_f = np.nanargmax(peak_E); print("EwEw:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
print(f"[Last  peak - {n_sweep3}]")
peak_E = np.ravel(spec.EuEu[n_sweep3]); peak_f = np.nanargmax(peak_E); print("EuEu:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EvEv[n_sweep3]); peak_f = np.nanargmax(peak_E); print("EvEv:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EwEw[n_sweep3]); peak_f = np.nanargmax(peak_E); print("EwEw:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))

### Stokes parameters (same with "sample")

In [ ]:
# NAN
if gap_mode == 1 and time_mode == 0:
    for i in range(spec.n_time-1):
        if data.scet[i+1]-data.scet[i] > 60:
            print("[gap]", i, data.epoch[i], i+1, data.epoch[i+1])
            juice_sid2.hf_sid2_spec_nan(data, i);   juice_sid2.hf_sid2_spec_nan(data, i+1)

# Stokes parameters
if comp_mode > 0:                           # [Complex]   0: Poweer  1: Matrix   3: Matrix-2D  >3: any
    spec = juice_spec.hf_getspec_angle (spec, 2)                            # Coherency & ange
    spec = juice_spec.hf_getspec_stokes(spec, 2)                            # Stokes and Polarization Parameters

In [ ]:
n_sweep = spec.n_time//2;  peak_E = np.ravel(spec.EuEu[n_sweep]); peak_f = np.nanargmax(peak_E); 
print(f"[SWEEP - {n_sweep}]  Peak at", '{:.1f}'.format(freq_1d[peak_f]), "kHz  (", peak_f, ")")
print("     EuEu:", '{:+.2e}'.format(np.ravel(spec.EuEu   [n_sweep])[peak_f]), spec.str_unit, "     EvEv:", '{:+.2e}'.format(np.ravel(spec.EvEv   [n_sweep])[peak_f]), spec.str_unit, "   EwEw:", '{:+.2e}'.format(np.ravel(spec.EwEw   [n_sweep])[peak_f]), spec.str_unit)
if comp_mode > 0:                           # [Complex]   0: Poweer  1: Matrix   3: Matrix-2D  >3: any
    print("  EuEv_re:", '{:+.2e}'.format(np.ravel(spec.EuEv_re[n_sweep])[peak_f]), spec.str_unit, "  EuEv_im:", '{:+.2e}'.format(np.ravel(spec.EuEv_im[n_sweep])[peak_f]), spec.str_unit, "  PHAuv:", '{:+5.1f} deg'.format(np.ravel(spec.E_PHAuv[n_sweep])[peak_f]), "  COHuv:", '{:+4.2f}'.format(np.ravel(spec.E_COHuv[n_sweep])[peak_f]), "  DoPuv:", '{:+4.2f}'.format(np.ravel(spec.E_DoPuv[n_sweep])[peak_f]), "  DoLuv:", '{:+4.2f}'.format(np.ravel(spec.E_DoLuv[n_sweep])[peak_f]), "  DoCuv:", '{:+4.2f}'.format(np.ravel(spec.E_DoCuv[n_sweep])[peak_f]), "  ANGuv:", '{:+5.1f} deg'.format(np.ravel(spec.E_ANGuv[n_sweep])[peak_f]))
    print("  EvEw_re:", '{:+.2e}'.format(np.ravel(spec.EvEw_re[n_sweep])[peak_f]), spec.str_unit, "  EvEw_im:", '{:+.2e}'.format(np.ravel(spec.EvEw_im[n_sweep])[peak_f]), spec.str_unit, "  PHAvw:", '{:+5.1f} deg'.format(np.ravel(spec.E_PHAvw[n_sweep])[peak_f]), "  COHvw:", '{:+4.2f}'.format(np.ravel(spec.E_COHvw[n_sweep])[peak_f]), "  DoPvw:", '{:+4.2f}'.format(np.ravel(spec.E_DoPvw[n_sweep])[peak_f]), "  DoLvw:", '{:+4.2f}'.format(np.ravel(spec.E_DoLvw[n_sweep])[peak_f]), "  DoCvw:", '{:+4.2f}'.format(np.ravel(spec.E_DoCvw[n_sweep])[peak_f]), "  ANGvw:", '{:+5.1f} deg'.format(np.ravel(spec.E_ANGvw[n_sweep])[peak_f]))
    print("  EwEu_re:", '{:+.2e}'.format(np.ravel(spec.EwEu_re[n_sweep])[peak_f]), spec.str_unit, "  EwEu_im:", '{:+.2e}'.format(np.ravel(spec.EwEu_im[n_sweep])[peak_f]), spec.str_unit, "  PHAwu:", '{:+5.1f} deg'.format(np.ravel(spec.E_PHAwu[n_sweep])[peak_f]), "  COHqu:", '{:+4.2f}'.format(np.ravel(spec.E_COHwu[n_sweep])[peak_f]), "  DoPwu:", '{:+4.2f}'.format(np.ravel(spec.E_DoPwu[n_sweep])[peak_f]), "  DoLwu:", '{:+4.2f}'.format(np.ravel(spec.E_DoLwu[n_sweep])[peak_f]), "  DoCwu:", '{:+4.2f}'.format(np.ravel(spec.E_DoCwu[n_sweep])[peak_f]), "  ANGwu:", '{:+5.1f} deg'.format(np.ravel(spec.E_ANGwu[n_sweep])[peak_f]))
    print('Stokes-UV: {:+.2e}'.format(np.ravel(spec.E_Iuv[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Quv[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Uuv[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Vuv[n_sweep])[peak_f]))
    print('Stokes-VW: {:+.2e}'.format(np.ravel(spec.E_Ivw[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Qvw[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Uvw[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Vvw[n_sweep])[peak_f]))
    print('Stokes-WU: {:+.2e}'.format(np.ravel(spec.E_Iwu[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Qwu[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Uwu[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Vwu[n_sweep])[peak_f]))

## Spectrum plot (same with "sample")

In [ ]:
EE_2d      = spec.EE.transpose();       EE_med      = np.nanmedian(spec.EE,      axis=0)
EuEu_2d    = spec.EuEu.transpose();     EuEu_med    = np.nanmedian(spec.EuEu,    axis=0)
EvEv_2d    = spec.EvEv.transpose();     EvEv_med    = np.nanmedian(spec.EvEv,    axis=0)
EwEw_2d    = spec.EwEw.transpose();     EwEw_med    = np.nanmedian(spec.EwEw,    axis=0)
if comp_mode > 0:                           # [Complex]   0: Poweer  1: Matrix   3: Matrix-2D  >3: any
    E_DoPuv_2d = spec.E_DoPuv.transpose();  E_DoPuv_med = np.nanmedian(spec.E_DoPuv, axis=0)
    E_DoPvw_2d = spec.E_DoPvw.transpose();  E_DoPvw_med = np.nanmedian(spec.E_DoPvw, axis=0)
    E_DoPwu_2d = spec.E_DoPwu.transpose();  E_DoPwu_med = np.nanmedian(spec.E_DoPwu, axis=0)
    E_DoLuv_2d = spec.E_DoLuv.transpose();  E_DoLuv_med = np.nanmedian(spec.E_DoLuv, axis=0)
    E_DoLvw_2d = spec.E_DoLvw.transpose();  E_DoLvw_med = np.nanmedian(spec.E_DoLvw, axis=0)
    E_DoLwu_2d = spec.E_DoLwu.transpose();  E_DoLwu_med = np.nanmedian(spec.E_DoLwu, axis=0)
    E_DoCuv_2d = spec.E_DoCuv.transpose();  E_DoCuv_med = np.nanmedian(spec.E_DoCuv, axis=0)
    E_DoCvw_2d = spec.E_DoCvw.transpose();  E_DoCvw_med = np.nanmedian(spec.E_DoCvw, axis=0)
    E_DoCwu_2d = spec.E_DoCwu.transpose();  E_DoCwu_med = np.nanmedian(spec.E_DoCwu, axis=0)
    E_ANGuv_2d = spec.E_ANGuv.transpose();  E_ANGuv_med = np.nanmedian(spec.E_ANGuv, axis=0)
    E_ANGvw_2d = spec.E_ANGvw.transpose();  E_ANGvw_med = np.nanmedian(spec.E_ANGvw, axis=0)
    E_ANGwu_2d = spec.E_ANGwu.transpose();  E_ANGwu_med = np.nanmedian(spec.E_ANGwu, axis=0)
    E_COHuv_2d = spec.E_COHuv.transpose();  E_COHuv_med = np.nanmedian(spec.E_COHuv, axis=0)
    E_COHvw_2d = spec.E_COHvw.transpose();  E_COHvw_med = np.nanmedian(spec.E_COHvw, axis=0)
    E_COHwu_2d = spec.E_COHwu.transpose();  E_COHwu_med = np.nanmedian(spec.E_COHwu, axis=0)
    E_PHAuv_2d = spec.E_PHAuv.transpose();  E_PHAuv_med = np.nanmedian(spec.E_PHAuv, axis=0)
    E_PHAvw_2d = spec.E_PHAvw.transpose();  E_PHAvw_med = np.nanmedian(spec.E_PHAvw, axis=0)
    E_PHAwu_2d = spec.E_PHAwu.transpose();  E_PHAwu_med = np.nanmedian(spec.E_PHAwu, axis=0)

In [ ]:
n_sweep1 = 0;  n_sweep2 = spec.n_time//2;  n_sweep3 = spec.n_time-1
p_min = p_min1;  p_max = p_max1
f_min = f_min1;  f_max = f_max1
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max

fig = plt.figure(figsize=(16, 11))
ax1 = fig.add_subplot(3, 1, 1);  ax2 = fig.add_subplot(3, 1, 2);  ax3 = fig.add_subplot(3, 1, 3)

ax1.plot(freq_1d, spec.EuEu[n_sweep1], '-r', linewidth=.9, label='EuEu 1st')
ax1.plot(freq_1d, spec.EuEu[n_sweep2], '-g', linewidth=.4, label='EuEu mid')
ax1.plot(freq_1d, spec.EuEu[n_sweep3], '-b', linewidth=.2, label='EuEu last')
ax2.plot(freq_1d, spec.EvEv[n_sweep1], '-r', linewidth=.9, label='EvEv 1st')
ax2.plot(freq_1d, spec.EvEv[n_sweep2], '-g', linewidth=.4, label='EvEv mid')
ax2.plot(freq_1d, spec.EvEv[n_sweep3], '-b', linewidth=.2, label='EvEv last')
ax3.plot(freq_1d, spec.EwEw[n_sweep1], '-r', linewidth=.9, label='EwEw 1st')
ax3.plot(freq_1d, spec.EwEw[n_sweep2], '-g', linewidth=.4, label='EwEw mid')
ax3.plot(freq_1d, spec.EwEw[n_sweep3], '-b', linewidth=.2, label='EwEw last')
ax1.set_yscale('log');      ax2.set_yscale('log');  ax3.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log');  ax3.set_xscale('log')
ax1.set_ylabel('EuEu '+spec.str_unit);     ax2.set_ylabel('EvEv '+spec.str_unit); ax3.set_ylabel('EwEw '+spec.str_unit);  ax3.set_xlabel('Frequency [kHz]')

date1 = spec.epoch[n_sweep1];  date1 = date1.strftime('1st: %Y/%m/%d %R:%S   ')
date2 = spec.epoch[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = spec.epoch[n_sweep3];  date3 = date3.strftime('Fin: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3;   ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)

xlim=[f_min, f_max]
ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim)
ylim=[10**(p_min), 10**(p_max)]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim)

fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-log.png'
    fig.savefig(png_fname)

In [ ]:
n_sweep1 = 0;  n_sweep2 = spec.n_time//2;  n_sweep3 = spec.n_time-1
p_min = p_min1;  p_max = p_max1
f_min = f_min1;  f_max = f_max1

fig = plt.figure(figsize=(16, 11));  ax1 = fig.add_subplot(1, 1, 1)
# ax1.plot(freq_1d, EE_med  ,      '-y',  linewidth=4.0, label='EE_med')
ax1.plot(freq_1d, EuEu_med,      '-r',  linewidth=0.9, label='EuEu_med')
ax1.plot(freq_1d, EvEv_med,      '-g',  linewidth=0.4, label='EvEv_med')
ax1.plot(freq_1d, EwEw_med,      '-b',  linewidth=0.2, label='EwEw_med')
"""
ax1.plot(freq_1d, spec.EuEu[n_sweep1], ':r',  linewidth=0.9, label='uu init')
ax1.plot(freq_1d, spec.EvEv[n_sweep1], ':g',  linewidth=0.4, label='vv init')
ax1.plot(freq_1d, spec.EwEw[n_sweep1], ':b',  linewidth=0.2, label='ww init')
ax1.plot(freq_1d, spec.EuEu[n_sweep2], '-.r', linewidth=0.9, label='uu half')
ax1.plot(freq_1d, spec.EvEv[n_sweep2], '-.g', linewidth=0.4, label='vv half')
ax1.plot(freq_1d, spec.EwEw[n_sweep2], '-.b', linewidth=0.2, label='ww half')
ax1.plot(freq_1d, spec.EuEu[n_sweep3], '-r',  linewidth=0.9, label='uu end')
ax1.plot(freq_1d, spec.EvEv[n_sweep3], '-g',  linewidth=0.4, label='vv end')
ax1.plot(freq_1d, spec.EwEw[n_sweep3], '-b',  linewidth=0.2, label='ww end')
"""
ax1.set_yscale('log');      ax2.set_yscale('log');  ax3.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log');  ax3.set_xscale('log')
ax1.set_ylabel('EuEu '+spec.str_unit);     ax2.set_ylabel('EvEv '+spec.str_unit); ax3.set_ylabel('EwEw '+spec.str_unit);  ax3.set_xlabel('Frequency [kHz]')

date1 = spec.epoch[n_sweep1];  date1 = date1.strftime('1st: %Y/%m/%d %R:%S   ')
date2 = spec.epoch[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = spec.epoch[n_sweep3];  date3 = date3.strftime('Fin: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3;   ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)

xlim=[f_min, f_max]
ax1.set_xlim(xlim)
ylim=[10**p_min, 10**p_max]; 
ax1.set_ylim(ylim)

fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec2.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec2-log.png'
    fig.savefig(png_fname)

In [ ]:
n_sweep1 = 0;  n_sweep2 = spec.n_time//2;  n_sweep3 = spec.n_time-1
p_min = p_min1;  p_max = p_max1
f_min = 0;       f_max = 5000

fig = plt.figure(figsize=(16, 11))
ax1 = fig.add_subplot(9, 1, 1);  ax2 = fig.add_subplot(9, 1, 2);  ax3 = fig.add_subplot(9, 1, 3);  ax4 = fig.add_subplot(9, 1, 4)
ax5 = fig.add_subplot(9, 1, 5);  ax6 = fig.add_subplot(9, 1, 6);  ax7 = fig.add_subplot(9, 1, 7);  ax8 = fig.add_subplot(9, 1, 8);  ax9 = fig.add_subplot(9, 1, 9)

ax1.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax1.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax1.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax1.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax2.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax2.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax2.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax2.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax3.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax3.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax3.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax3.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax4.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax4.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax4.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax4.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax5.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax5.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax5.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax5.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax6.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax6.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax6.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax6.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax7.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax7.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax7.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax7.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax8.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax8.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax8.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax8.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax9.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax9.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax9.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax9.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax1.set_yscale('log');  ax2.set_yscale('log');  ax3.set_yscale('log');  ax4.set_yscale('log');  ax5.set_yscale('log')
ax6.set_yscale('log');  ax7.set_yscale('log');  ax8.set_yscale('log');  ax9.set_yscale('log')

ax1.set_ylabel(spec.str_unit); ax2.set_ylabel(spec.str_unit); ax3.set_ylabel(spec.str_unit); ax4.set_ylabel(spec.str_unit); ax5.set_ylabel(spec.str_unit)
ax6.set_ylabel(spec.str_unit); ax7.set_ylabel(spec.str_unit); ax8.set_ylabel(spec.str_unit); ax9.set_ylabel(spec.str_unit)
ax9.set_xlabel('Frequency [kHz]')
date1 = spec.epoch[n_sweep1];  date1 = date1.strftime(f'1st {n_sweep1}: %Y/%m/%d %R:%S ')
date2 = spec.epoch[n_sweep2];  date2 = date2.strftime(f'Mid {n_sweep2}: %Y/%m/%d %R:%S ')
date3 = spec.epoch[n_sweep3];  date3 = date3.strftime(f'Fin {n_sweep3}: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3;  ax1.set_title(title_date)

ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);  ax6.legend(loc='upper right', fontsize=8)
ax7.legend(loc='upper right', fontsize=8);  ax8.legend(loc='upper right', fontsize=8);  ax9.legend(loc='upper right', fontsize=8)

xlim=[f_min,       f_max];        ax1.set_xlim(xlim); xlim=[f_min+5000,  f_max+5000];   ax2.set_xlim(xlim)
xlim=[f_min+10000, f_max+10000];  ax3.set_xlim(xlim); xlim=[f_min+15000, f_max+15000];  ax4.set_xlim(xlim)
xlim=[f_min+20000, f_max+20000];  ax5.set_xlim(xlim); xlim=[f_min+25000, f_max+25000];  ax6.set_xlim(xlim)
xlim=[f_min+30000, f_max+30000];  ax7.set_xlim(xlim); xlim=[f_min+35000, f_max+35000];  ax8.set_xlim(xlim)
xlim=[f_min+40000, f_max+40000];  ax9.set_xlim(xlim)
ax1.tick_params(labelsize=7); ax2.tick_params(labelsize=7); ax3.tick_params(labelsize=7); ax4.tick_params(labelsize=7); ax5.tick_params(labelsize=7)
ax6.tick_params(labelsize=7); ax7.tick_params(labelsize=7); ax8.tick_params(labelsize=7); ax9.tick_params(labelsize=7)

ylim=[10**p_min, 10**p_max]
ax1.set_ylim(ylim); ax2.set_ylim(ylim); ax3.set_ylim(ylim); ax4.set_ylim(ylim); ax5.set_ylim(ylim); ax6.set_ylim(ylim);  
ax7.set_ylim(ylim); ax8.set_ylim(ylim); ax9.set_ylim(ylim)

fig.subplots_adjust(hspace=0.2)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-fine.png'
    fig.savefig(png_fname)

In [ ]:
p_min = p_min1;  p_max = p_max1
f_min = f_min1;  f_max = f_max1
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max;  print(data.frequency[0][0][0], data.frequency[0][8][0])

fig2d = plt.figure(figsize=[16,11])
if time_mode == 1:
    ax1 = fig2d.add_subplot(4, 1, 1); ax2 = fig2d.add_subplot(4, 1, 2); ax3 = fig2d.add_subplot(4, 1, 3); ax4.set_xlabel(str_date); ax4 = fig2d.add_subplot(4, 1, 4)
else:
    ax1 = fig2d.add_subplot(3, 1, 1); ax2 = fig2d.add_subplot(3, 1, 2); ax3 = fig2d.add_subplot(3, 1, 3); ax3.set_xlabel(str_date)

ax1.set_ylabel('EuEu [kHz]'); ax2.set_ylabel('EvEv [kHz]'); ax3.set_ylabel('EwEw [kHz]')
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log')

if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.plot(np.ravel(spec.epoch[:]), '.')
    pp3 = fig2d.colorbar(p3, ax=ax4, orientation="vertical"); pp3.set_label(spec.str_unit)
else:
    p1 = ax1.pcolormesh(spec.epoch, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(spec.epoch, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(spec.epoch, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(spec.str_unit)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical"); pp2.set_label(spec.str_unit)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp3.set_label(spec.str_unit)

print( "(", num_1d[0],")", spec.epoch[0], "-", "(", num_1d[-1],")", spec.epoch[-1] )
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]; ax4.set_xlim(xlim) 
else:
    if t_min0 == 0:
        xlim=[spec.epoch[0], spec.epoch[-1]]
    else:
        xlim=[t_min0, t_max0]
    # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
    # E_max = '2024-08-21 05:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
    print("==>", xlim)
ax1.set_xlim(xlim);  ax2.set_xlim(xlim); ax3.set_xlim(xlim)

plt.subplots_adjust(hspace=.03)
plt.show
if dump_mode > 0:
    png_fname = work_dir+data_name+'_FT.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-log.png'
    fig2d.savefig(png_fname)

In [ ]:
if comp_mode > 0:                           # [Complex]   0: Poweer  1: Matrix   3: Matrix-2D  >3: any
    p_min = p_min1;  p_max = p_max1
    f_min = f_min1;  f_max = f_max1
    if f_mode == 0:
        f_min = f_mode_min;  f_max = f_mode_max

    fig2d = plt.figure(figsize=[16,11])
    if time_mode == 1:
        ax1 = fig2d.add_subplot(10, 1, 1); ax2 = fig2d.add_subplot(10, 1, 2); ax3 = fig2d.add_subplot(10, 1, 3)
        ax4 = fig2d.add_subplot(10, 1, 4); ax5 = fig2d.add_subplot(10, 1, 5); ax6 = fig2d.add_subplot(10, 1, 6)
        ax7 = fig2d.add_subplot(10, 1, 7); ax8 = fig2d.add_subplot(10, 1, 8); ax9 = fig2d.add_subplot(10, 1, 9); ax10 = fig2d.add_subplot(10, 1, 10)
    else:
        ax1 = fig2d.add_subplot(9, 1, 1);  ax2 = fig2d.add_subplot(9, 1, 2);  ax3 = fig2d.add_subplot(9, 1, 3)
        ax4 = fig2d.add_subplot(9, 1, 4);  ax5 = fig2d.add_subplot(9, 1, 5);  ax6 = fig2d.add_subplot(9, 1, 6)
        ax7 = fig2d.add_subplot(9, 1, 7);  ax8 = fig2d.add_subplot(9, 1, 8);  ax9 = fig2d.add_subplot(9, 1, 9)
    ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max); ax5.set_ylim(f_min, f_max)
    ax6.set_ylim(f_min, f_max); ax7.set_ylim(f_min, f_max); ax8.set_ylim(f_min, f_max); ax9.set_ylim(f_min, f_max)
    if f_mode == 1:
        ax1.set_yscale('log');    ax2.set_yscale('log');     ax3.set_yscale('log');     ax4.set_yscale('log');     ax5.set_yscale('log')
        ax6.set_yscale('log');    ax7.set_yscale('log');     ax8.set_yscale('log');     ax9.set_yscale('log')
    ax1.set_ylabel('Lin. uv');    ax2.set_ylabel('Lin. vw'); ax3.set_ylabel('Lin. wu'); ax4.set_ylabel('Cir. uv'); ax5.set_ylabel('Cir. vw');  
    ax6.set_ylabel('Cir. wu');    ax7.set_ylabel('Ang uv');  ax8.set_ylabel('Ang vw');  ax9.set_ylabel('Ang wu')
    ax1.set_title("[" + data_name + "]")
    if time_mode == 1: ax10.set_xlabel(str_date)
    else:              ax9.set_xlabel(str_date)

    if time_mode == 1:
        p1 = ax1.pcolormesh(num_1d, freq_1d, E_DoLuv_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p2 = ax2.pcolormesh(num_1d, freq_1d, E_DoLvw_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p3 = ax3.pcolormesh(num_1d, freq_1d, E_DoLwu_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p4 = ax4.pcolormesh(num_1d, freq_1d, E_DoCuv_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p5 = ax5.pcolormesh(num_1d, freq_1d, E_DoCvw_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p6 = ax6.pcolormesh(num_1d, freq_1d, E_DoCwu_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p7 = ax7.pcolormesh(num_1d, freq_1d, E_ANGuv_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p8 = ax8.pcolormesh(num_1d, freq_1d, E_ANGvw_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p9 = ax9.pcolormesh(num_1d, freq_1d, E_ANGwu_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p10 = ax10.plot(np.ravel(data.epoch[:]), '.')
        pp9 = fig2d.colorbar(p9, ax=ax10, orientation="vertical"); pp9.set_label(spec.str_unit)
    else:
        p1 = ax1.pcolormesh(spec.epoch, freq_1d, E_DoLuv_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p2 = ax2.pcolormesh(spec.epoch, freq_1d, E_DoLvw_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p3 = ax3.pcolormesh(spec.epoch, freq_1d, E_DoLwu_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p4 = ax4.pcolormesh(spec.epoch, freq_1d, E_DoCuv_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p5 = ax5.pcolormesh(spec.epoch, freq_1d, E_DoCvw_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p6 = ax6.pcolormesh(spec.epoch, freq_1d, E_DoCwu_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p7 = ax7.pcolormesh(spec.epoch, freq_1d, E_ANGuv_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p8 = ax8.pcolormesh(spec.epoch, freq_1d, E_ANGvw_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p9 = ax9.pcolormesh(spec.epoch, freq_1d, E_ANGwu_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
    pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical")
    pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical")
    pp5 = fig2d.colorbar(p5, ax=ax5, orientation="vertical"); pp6 = fig2d.colorbar(p6, ax=ax6, orientation="vertical")
    pp7 = fig2d.colorbar(p7, ax=ax7, orientation="vertical"); pp8 = fig2d.colorbar(p8, ax=ax8, orientation="vertical")
    pp9 = fig2d.colorbar(p9, ax=ax9, orientation="vertical"); pp7.set_label("[deg]"); pp8.set_label("[deg]"); pp9.set_label("[deg]")

    print( "(", num_1d[0],")", spec.epoch[0], "-", "(", num_1d[-1],")", spec.epoch[-1] )
    if time_mode == 1:
        xlim=[num_1d[0], num_1d[-1]]; ax10.set_xlim(xlim) 
    else:
        if t_min0 == 0:
            xlim=[spec.epoch[0], spec.epoch[-1]]
        else:
            xlim=[t_min0, t_max0]
        # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        # E_max = '2024-08-21 05:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
        print("==>", xlim)
    ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)
    ax6.set_xlim(xlim); ax7.set_xlim(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)

    plt.subplots_adjust(hspace=.05)
    plt.show()
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_AngT.png'
        if f_mode == 1:
            png_fname = work_dir+data_name+'_AngT-log.png'
        fig2d.savefig(png_fname)

# CAL Spectrum

In [ ]:
# CAL line pickup
peaks,_ = find_peaks(EuEu_med,                   height=1e5 * 10**(spec.cf/10), distance=96)
print('CAL:', '{:.1f}'.format(spec.cf), '{:.1e}'.format(1e5 * 10**(spec.cf/10)), "  [CAL lines]", peaks.shape[0])

In [ ]:
CAL_f_gain, CAL_f_phase = juice_cal.spec_gain_phase(freq_1d, unit_mode, T_HF, T_RWI)

p_min = p_min1;  p_max = p_max1
f_min = f_min1;  f_max = f_max1
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max;  print(data.frequency[0][0][0], data.frequency[0][8][0])

fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(7, 1, 1);  ax2 = fig.add_subplot(7, 1, 2);  ax3 = fig.add_subplot(7, 1, 3);  ax4 = fig.add_subplot(7, 1, 4)
ax5 = fig.add_subplot(7, 1, 5);  ax6 = fig.add_subplot(7, 1, 6);  ax7 = fig.add_subplot(7, 1, 7)

ax1.plot(freq_1d, CAL_f_gain[0],  '-r', label='cal table [U]', linewidth=1.6)
ax1.plot(freq_1d, CAL_f_gain[1],  '-g', label='cal table [V]', linewidth=1.6)
ax1.plot(freq_1d, CAL_f_gain[2],  '-b', label='cal table [W]', linewidth=1.6)
ax2.plot(freq_1d, CAL_f_phase[0], '-r', label='cal table [U]', linewidth=1.6)
ax2.plot(freq_1d, CAL_f_phase[1], '-g', label='cal table [V]', linewidth=1.6)
ax2.plot(freq_1d, CAL_f_phase[2], '-b', label='cal table [W]', linewidth=1.6)
ax3.plot(freq_1d[peaks],  EuEu_med[peaks],                      '-r', label='U peak',    linewidth=1)
ax3.plot(freq_1d[peaks],  EvEv_med[peaks],                      '-g', label='V peak',    linewidth=1)
ax3.plot(freq_1d[peaks],  EwEw_med[peaks],                      '-b', label='W peak',    linewidth=1)
ax4.plot(freq_1d[peaks], (EuEu_med[peaks]/EuEu_med[peaks])**.5, '-r', label='U/U',       linewidth=1)
ax4.plot(freq_1d[peaks], (EvEv_med[peaks]/EuEu_med[peaks])**.5, '-g', label='V/U',       linewidth=1)
ax4.plot(freq_1d[peaks], (EwEw_med[peaks]/EuEu_med[peaks])**.5, '-b', label='W/U',       linewidth=1)
if comp_mode > 0:                           # [Complex]   0: Poweer  1: Matrix   3: Matrix-2D  >3: any
    ax5.plot(freq_1d[peaks],  E_PHAuv_med[peaks],                   '-g', label='Phase_V-U', linewidth=1)
    ax5.plot(freq_1d[peaks], -E_PHAwu_med[peaks],                   '-b', label='Phase_W-U', linewidth=1)
    ax5.plot(freq_1d[peaks],  E_PHAvw_med[peaks],                   '-k', label='Phase_W-V', linewidth=1)
    ax6.plot(freq_1d[peaks],  E_COHuv_med[peaks],                   '-r', label='E_COHuv',   linewidth=1)
    ax6.plot(freq_1d[peaks],  E_COHvw_med[peaks],                   '-g', label='E_COHvw',   linewidth=1)
    ax6.plot(freq_1d[peaks],  E_COHwu_med[peaks],                   '-b', label='E_COHwu',   linewidth=1)
    ax7.plot(freq_1d[peaks],  E_DoCuv_med[peaks],                   '-r', label='E_DoCuv',   linewidth=1)
    ax7.plot(freq_1d[peaks],  E_DoCvw_med[peaks],                   '-g', label='E_DoCvw',   linewidth=1)
    ax7.plot(freq_1d[peaks],  E_DoCwu_med[peaks],                   '-b', label='E_DoCuw',   linewidth=1)
ax1.set_yscale('log');                        ax3.set_yscale('log')
ax1.set_xscale('log'); ax2.set_xscale('log'); ax3.set_xscale('log'); ax4.set_xscale('log'); ax5.set_xscale('log'); ax6.set_xscale('log'); ax7.set_xscale('log')

ax1.set_ylabel('CAL gain'); ax2.set_ylabel('CAL phase'); ax3.set_ylabel('EE'); ax4.set_ylabel('EE ratio'); ax5.set_ylabel('Relative Phase [deg]')
ax6.set_ylabel('COH');  ax7.set_ylabel('DoC');   ax7.set_xlabel('Frequency [kHz]')
ax1.legend(loc='upper left', fontsize=8); ax2.legend(loc='upper left', fontsize=8); ax3.legend(loc='upper left', fontsize=8)
ax4.legend(loc='upper left', fontsize=8); ax5.legend(loc='upper left', fontsize=8); ax6.legend(loc='upper left', fontsize=8)
ax7.legend(loc='upper left', fontsize=8)
title_label = '[CAL] unit-mode:' + format(unit_mode, '1') + '  band-mode:' + format(band_mode, '1');  ax1.set_title(title_label)

# range: X-axis
xlim=[f_min, f_max]; ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim);  ax7.set_xlim(xlim)
# range: Y-axis
# ylim=[10**(p_min), 10**(p_max)];  ax1.set_ylim(ylim)

fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'-cal_spec-cal.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'-cal_spec-cal-log.png'
    fig.savefig(png_fname)

# CAL table making

In [ ]:
peaks_EE,_ = find_peaks(EuEu_med,                height=1e5 * 10**(spec.cf/10), distance=96)
print('CAL:', '{:.1f}'.format(spec.cf), '{:.1e}'.format(1e5 * 10**(spec.cf/10)), "  [CAL lines]", peaks_EE.shape[0])
n_cal       = peaks_EE.shape[0]
CAL_freq    = freq_1d[peaks_EE];                                    # print(CAL_freq)
CAL_gain_u  = (EuEu_med[peaks_EE] / EuEu_med[peaks_EE])**.5         # print(CAL_gain_u)
CAL_gain_v  = (EvEv_med[peaks_EE] / EuEu_med[peaks_EE])**.5         # print(CAL_gain_v)
CAL_gain_w  = (EwEw_med[peaks_EE] / EuEu_med[peaks_EE])**.5         # print(CAL_gain_w)
if comp_mode > 0:                           # [Complex]   0: Poweer  1: Matrix   3: Matrix-2D  >3: any
    CAL_phase_u =  E_PHAuv_med[peaks_EE] - E_PHAuv_med[peaks_EE]        # print(CAL_phase_u)
    CAL_phase_v =  E_PHAuv_med[peaks_EE]                                # print(CAL_phase_v)
    CAL_phase_w = -E_PHAwu_med[peaks_EE]                                # print(CAL_phase_w)

In [ ]:
p_min = p_min1;  p_max = p_max1
f_min = f_min1;  f_max = f_max1
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max;  print(data.frequency[0][0][0], data.frequency[0][8][0])

fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(freq_1d[peaks_EE], EuEu_med[peaks_EE],              '--r', label='U',     linewidth=.5)
ax1.plot(freq_1d[peaks_EE], EvEv_med[peaks_EE],              '--g', label='V',     linewidth=.5)
ax1.plot(freq_1d[peaks_EE], EwEw_med[peaks_EE],              '--b', label='W',     linewidth=.5)
ax1.plot(freq_1d[peaks_EE], EuEu_med[peaks_EE]/CAL_gain_u**2, '-r', label='U cal', linewidth=1)
ax1.plot(freq_1d[peaks_EE], EvEv_med[peaks_EE]/CAL_gain_v**2, '-g', label='V cal', linewidth=1)
ax1.plot(freq_1d[peaks_EE], EwEw_med[peaks_EE]/CAL_gain_w**2, '-b', label='W cal', linewidth=1)
if unit_mode == 3 and band_mode == 0:
    CAL_gain = juice_cal.spec_gain_onboard_cal(freq_1d[peaks_EE])       # CAL-line gain
    ax1.plot(freq_1d[peaks_EE], EuEu_med[peaks_EE]*CAL_gain**2,        '.r', label='CAL corr', linewidth=1)
    ax2.plot(freq_1d[peaks_EE], EuEu_med[peaks_EE]*CAL_gain**2/0.1**2, '.r', label='CAL corr', linewidth=1)
ax2.plot(freq_1d[peaks_EE], CAL_gain_u,                       '-r', label='U/U',   linewidth=1)
ax2.plot(freq_1d[peaks_EE], CAL_gain_v,                       '-g', label='V/U',   linewidth=1)
ax2.plot(freq_1d[peaks_EE], CAL_gain_w,                       '-b', label='W/U',   linewidth=1)
if comp_mode > 0:                           # [Complex]   0: Poweer  1: Matrix   3: Matrix-2D  >3: any
    ax3.plot(freq_1d[peaks_EE], E_COHuv_med[peaks_EE],           '.r', label='COHuv', linewidth=.5)
    ax3.plot(freq_1d[peaks_EE], E_COHvw_med[peaks_EE],           '.g', label='COHvw', linewidth=.5)
    ax3.plot(freq_1d[peaks_EE], E_COHwu_med[peaks_EE],           '.b', label='COHwu', linewidth=.5)
    ax3.plot(freq_1d[peaks_EE], E_DoCuv_med[peaks_EE],           '-r', label='DoCuv',  linewidth=.5)
    ax3.plot(freq_1d[peaks_EE], E_DoCvw_med[peaks_EE],           '-g', label='DoCvw',  linewidth=.5)
    ax3.plot(freq_1d[peaks_EE], E_DoCwu_med[peaks_EE],           '-b', label='DoCuw',  linewidth=.5)
    ax4.plot(freq_1d[peaks_EE], CAL_phase_u,                     '-r', label='Phase U-U - cal')
    ax4.plot(freq_1d[peaks_EE], CAL_phase_v,                     '-g', label='Phase V-U - cal')
    ax4.plot(freq_1d[peaks_EE], CAL_phase_w,                     '-b', label='Phase W-U - cal')

ax1.set_yscale('log')
ax1.set_xscale('log');  ax2.set_xscale('log');  ax3.set_xscale('log');  ax4.set_xscale('log')
ax1.set_title("[" + data_name + "]  ")

# Label
ax1.set_ylabel('EE'); ax2.set_ylabel('EE ratio'); ax3.set_ylabel('DoC'); ax4.set_ylabel('ANG');  ax4.set_xlabel('Frequency [kHz]')
ax1.legend(loc='upper left', fontsize=8); ax2.legend(loc='upper left', fontsize=8); ax3.legend(loc='upper left', fontsize=8); ax4.legend(loc='upper left', fontsize=8)

# range: X-axis
xlim=[f_min, f_max]
ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim)
# range: Y-axis
# ylim=[10**(p_min), 10**(p_max)];  ax1.set_ylim(ylim)

fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'-cal_table.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'-cal_table-log.png'
    fig.savefig(png_fname)

In [ ]:
p_min = p_min1;  p_max = p_max1
f_min = f_min1;  f_max = f_max1
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max;  print(data.frequency[0][0][0], data.frequency[0][8][0])

fig2d = plt.figure(figsize=[16,11])
if time_mode == 1:
    ax1 = fig2d.add_subplot(4, 1, 1); ax2 = fig2d.add_subplot(4, 1, 2); ax3 = fig2d.add_subplot(4, 1, 3); ax4 = fig2d.add_subplot(4, 1, 4); ax4.set_xlabel(str_date)
else:
    ax1 = fig2d.add_subplot(3, 1, 1); ax2 = fig2d.add_subplot(3, 1, 2); ax3 = fig2d.add_subplot(3, 1, 3);                                   ax3.set_xlabel(str_date)

if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d[peaks_EE], EuEu_2d[:][peaks_EE], norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d[peaks_EE], EvEv_2d[:][peaks_EE], norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d[peaks_EE], EwEw_2d[:][peaks_EE], norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.plot(np.ravel(spec.epoch[:]), '.')
    pp3 = fig2d.colorbar(p3, ax=ax4, orientation="vertical"); pp3.set_label(spec.str_unit)
else:
    p1 = ax1.pcolormesh(spec.epoch, freq_1d[peaks_EE], EuEu_2d[:][peaks_EE], norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(spec.epoch, freq_1d[peaks_EE], EvEv_2d[:][peaks_EE], norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(spec.epoch, freq_1d[peaks_EE], EwEw_2d[:][peaks_EE], norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(spec.str_unit)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical"); pp2.set_label(spec.str_unit)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp3.set_label(spec.str_unit)

ax1.set_ylabel('CAL EuEu'); ax2.set_ylabel('CAL EvEv'); ax3.set_ylabel('CAL EwEw')
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');  ax2.set_yscale('log');  ax3.set_yscale('log')

print( "(", num_1d[0],")", spec.epoch[0], "-", "(", num_1d[-1],")", spec.epoch[-1] )
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]; ax4.set_xlim(xlim) 
else:
    if t_min0 == 0:
        xlim=[spec.epoch[0], spec.epoch[-1]]
    else:
        xlim=[t_min0, t_max0]
    # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
    # E_max = '2024-08-21 05:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
    print("==>", xlim)
ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim)

ax1.set_title("[" + data_name + "]  ")

plt.subplots_adjust(hspace=0.03)
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'-cal_table-FT.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'-cal_table-FT-log.png'
    fig2d.savefig(png_fname)

In [ ]:
p_min = p_min1;  p_max = p_max1
f_min = f_min1;  f_max = f_max1
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max

fig2d = plt.figure(figsize=[16,11])
if time_mode == 1:
    ax1 = fig2d.add_subplot(10, 1, 1); ax2 = fig2d.add_subplot(10, 1, 2); ax3 = fig2d.add_subplot(10, 1, 3)
    ax4 = fig2d.add_subplot(10, 1, 4); ax5 = fig2d.add_subplot(10, 1, 5); ax6 = fig2d.add_subplot(10, 1, 6)
    ax7 = fig2d.add_subplot(10, 1, 7); ax8 = fig2d.add_subplot(10, 1, 8); ax9 = fig2d.add_subplot(10, 1, 9); ax10 = fig2d.add_subplot(10, 1, 10)
else:
    ax1 = fig2d.add_subplot(9, 1, 1);  ax2 = fig2d.add_subplot(9, 1, 2);  ax3 = fig2d.add_subplot(9, 1, 3)
    ax4 = fig2d.add_subplot(9, 1, 4);  ax5 = fig2d.add_subplot(9, 1, 5);  ax6 = fig2d.add_subplot(9, 1, 6)
    ax7 = fig2d.add_subplot(9, 1, 7);  ax8 = fig2d.add_subplot(9, 1, 8);  ax9 = fig2d.add_subplot(9, 1, 9)
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max); ax5.set_ylim(f_min, f_max)
ax6.set_ylim(f_min, f_max); ax7.set_ylim(f_min, f_max); ax8.set_ylim(f_min, f_max); ax9.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log');        ax4.set_yscale('log');        ax5.set_yscale('log')
    ax6.set_yscale('log');    ax7.set_yscale('log');        ax8.set_yscale('log');        ax9.set_yscale('log')
ax1.set_ylabel('Lin. uv');    ax2.set_ylabel('Lin. vw');    ax3.set_ylabel('Lin. wu');    ax4.set_ylabel('Cir. uv');    ax5.set_ylabel('Cir. vw');  
ax6.set_ylabel('Cir. wu');    ax7.set_ylabel('Ang uv');     ax8.set_ylabel('Ang vw');     ax9.set_ylabel('Ang wu')
ax1.set_title("[" + data_name + "]  ");     
if time_mode == 1:
    ax10.set_xlabel(str_date)
else:
    ax9.set_xlabel(str_date)

if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d[peaks_EE], E_DoLuv_2d[:][peaks_EE], norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d[peaks_EE], E_DoLvw_2d[:][peaks_EE], norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d[peaks_EE], E_DoLwu_2d[:][peaks_EE], norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
    p4 = ax4.pcolormesh(num_1d, freq_1d[peaks_EE], E_DoCuv_2d[:][peaks_EE], norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
    p5 = ax5.pcolormesh(num_1d, freq_1d[peaks_EE], E_DoCvw_2d[:][peaks_EE], norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
    p6 = ax6.pcolormesh(num_1d, freq_1d[peaks_EE], E_DoCwu_2d[:][peaks_EE], norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
    p7 = ax7.pcolormesh(num_1d, freq_1d[peaks_EE], E_ANGuv_2d[:][peaks_EE], norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
    p8 = ax8.pcolormesh(num_1d, freq_1d[peaks_EE], E_ANGvw_2d[:][peaks_EE], norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
    p9 = ax9.pcolormesh(num_1d, freq_1d[peaks_EE], E_ANGwu_2d[:][peaks_EE], norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
    p10 = ax10.plot(np.ravel(data.epoch[:]), '.')
    pp9 = fig2d.colorbar(p9, ax=ax10, orientation="vertical"); pp9.set_label(spec.str_unit)
else:
    p1 = ax1.pcolormesh(spec.epoch, freq_1d[peaks_EE], E_DoLuv_2d[:][peaks_EE], norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
    p2 = ax2.pcolormesh(spec.epoch, freq_1d[peaks_EE], E_DoLvw_2d[:][peaks_EE], norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
    p3 = ax3.pcolormesh(spec.epoch, freq_1d[peaks_EE], E_DoLwu_2d[:][peaks_EE], norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
    p4 = ax4.pcolormesh(spec.epoch, freq_1d[peaks_EE], E_DoCuv_2d[:][peaks_EE], norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
    p5 = ax5.pcolormesh(spec.epoch, freq_1d[peaks_EE], E_DoCvw_2d[:][peaks_EE], norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
    p6 = ax6.pcolormesh(spec.epoch, freq_1d[peaks_EE], E_DoCwu_2d[:][peaks_EE], norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
    p7 = ax7.pcolormesh(spec.epoch, freq_1d[peaks_EE], E_ANGuv_2d[:][peaks_EE], norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
    p8 = ax8.pcolormesh(spec.epoch, freq_1d[peaks_EE], E_ANGvw_2d[:][peaks_EE], norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
    p9 = ax9.pcolormesh(spec.epoch, freq_1d[peaks_EE], E_ANGwu_2d[:][peaks_EE], norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical")
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical")
pp5 = fig2d.colorbar(p5, ax=ax5, orientation="vertical"); pp6 = fig2d.colorbar(p6, ax=ax6, orientation="vertical")
pp7 = fig2d.colorbar(p7, ax=ax7, orientation="vertical"); pp8 = fig2d.colorbar(p8, ax=ax8, orientation="vertical")
pp9 = fig2d.colorbar(p9, ax=ax9, orientation="vertical")

print( "(", num_1d[0],")", spec.epoch[0], "-", "(", num_1d[-1],")", spec.epoch[-1] )
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]; ax10.set_xlim(xlim) 
else:
    if t_min0 == 0:
        xlim=[spec.epoch[0], spec.epoch[-1]]
    else:
        xlim=[t_min0, t_max0]
    # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
    # E_max = '2024-08-21 05:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
    print("==>", xlim)
ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)
ax6.set_xlim(xlim); ax7.set_xlim(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)

plt.subplots_adjust(hspace=0.03)
plt.show()
if dump_mode == 1:
    png_fname = work_dir+data_name+'-cal_table-AngT.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'-cal_table-AngT-log.png'
    fig2d.savefig(png_fname)

## Write cal table

In [ ]:
if CAL_make_mode > 0:
    # CAL CSV write
    cal_dir  = './lib/';  cal_name = 'hf_OnboardCAL.csv';  cal_file = cal_dir + cal_name
    with open(cal_file, 'w') as f:
        writer = csv.writer(f)
        for i in range(n_cal):
            writer.writerow([ i+1, freq_1d[peaks_EE[i]], 
                            EuEu_med[peaks_EE[i]],    EvEv_med[peaks_EE[i]],    EwEw_med[peaks_EE[i]], 
                            E_PHAuv_med[peaks_EE[i]], E_PHAvw_med[peaks_EE[i]], E_PHAwu_med[peaks_EE[i]],
                            np.log10(CAL_gain_u[i])*20, CAL_phase_u[i], 
                            np.log10(CAL_gain_v[i])*20, CAL_phase_v[i], 
                            np.log10(CAL_gain_w[i])*20, CAL_phase_w[i], ])

# CAL table -- TEST

In [ ]:
cal_test = 1

In [ ]:
if cal_test>0:
    T_RWI = [-175, 20, 75]
    T_HF  = [-30,  20,  75]

    for j in range (4):
        for i in range(3):
            wave_cal = juice_cal.wave_cal(data, 2, j, T_HF[1], T_RWI[i])
            print("unit_mode:", wave_cal.str_unit, j, "T-RWI:", T_RWI[i], "->", wave_cal.cf)
        for i in range(3):
            wave_cal = juice_cal.wave_cal(data, 2, j, T_HF[i], T_RWI[1])
            print("unit_mode:", wave_cal.str_unit, j, "T-HF:", T_HF[i], "->", wave_cal.cf)        

In [ ]:
if cal_test>0:
    freq = np.arange(5000)*10 + 20;  n_freq = freq.shape[0]
    T_HF  = 25;  T_RWI = 25
    CAL_f_gain  = np.zeros((3, n_freq));  CAL_f_phase = np.zeros((3, n_freq))

    fig = plt.figure(figsize=(14, 8))
    ax1 = fig.add_subplot(2, 1, 1);  ax2 = fig.add_subplot(2, 1, 2);  

    # Harness loss 
    CAL_gain = juice_cal.spec_gain_onboard_cal(freq)
    ax1.plot(freq, CAL_gain,       '.k',  label='CAL linw', linewidth=0.3)
    # Harness loss 
 
    CAL_f_gain, CAL_f_phase = juice_cal.spec_gain_phase(freq, 1, T_HF, T_RWI)
    ax1.plot(freq, CAL_f_gain[0],  '--r', label='ADC [U]', linewidth=0.3)
    ax1.plot(freq, CAL_f_gain[1],  '--g', label='ADC [V]', linewidth=0.3)
    ax1.plot(freq, CAL_f_gain[2],  '--b', label='ADC [W]', linewidth=0.3)
    ax2.plot(freq, CAL_f_phase[0], '--r', label='ADC [U]', linewidth=0.3)
    ax2.plot(freq, CAL_f_phase[1], '--g', label='ADC [V]', linewidth=0.3)
    ax2.plot(freq, CAL_f_phase[2], '--b', label='ADC [W]', linewidth=0.3)
    CAL_f_gain, CAL_f_phase = juice_cal.spec_gain_phase(freq, 2, T_HF, T_RWI)
    ax1.plot(freq, CAL_f_gain[0],  '-r', label='HF [U]', linewidth=0.6)
    ax1.plot(freq, CAL_f_gain[1],  '-g', label='HF [V]', linewidth=0.6)
    ax1.plot(freq, CAL_f_gain[2],  '-b', label='HF [W]', linewidth=0.6)
    ax2.plot(freq, CAL_f_phase[0], '-r', label='HF [U]', linewidth=0.6)
    ax2.plot(freq, CAL_f_phase[1], '-g', label='HF [V]', linewidth=0.6)
    ax2.plot(freq, CAL_f_phase[2], '-b', label='HF [W]', linewidth=0.6)
    CAL_f_gain, CAL_f_phase = juice_cal.spec_gain_phase(freq, 3, T_HF, T_RWI)
    ax1.plot(freq, CAL_f_gain[0],  '.r', label='RWI [U]', linewidth=1.6)
    ax1.plot(freq, CAL_f_gain[1],  '.g', label='RWI [V]', linewidth=1.6)
    ax1.plot(freq, CAL_f_gain[2],  '.b', label='RWI [W]', linewidth=1.6)
    ax2.plot(freq, CAL_f_phase[0], '.r', label='RWI [U]', linewidth=1.6)
    ax2.plot(freq, CAL_f_phase[1], '.g', label='RWI [V]', linewidth=1.6)
    ax2.plot(freq, CAL_f_phase[2], '.b', label='RWI [W]', linewidth=1.6)

    ax1.set_xscale('log');  ax2.set_xscale('log');  ax1.set_yscale('log')
    ax1.set_ylabel('gain'); ax2.set_ylabel('phase')
    ax1.legend(loc='upper center', fontsize=8);  ax2.legend(loc='upper center', fontsize=8)
    xlim=[20, 45000]; ax1.set_xlim(xlim); ax2.set_xlim(xlim)
    fig.subplots_adjust(hspace=0)
    fig.show

In [ ]:
if cal_test>0:
    CAL_f_mode = 3

    freq = np.arange(5000)*10 + 20;  n_freq = freq.shape[0]
    T_HF  = 25;  T_RWI = 25
    CAL_f_gain  = np.zeros((3, n_freq));  CAL_f_phase = np.zeros((3, n_freq))
    CAL_f_gain, CAL_f_phase = juice_cal.spec_gain_phase(freq, CAL_f_mode, T_HF, T_RWI)

    fig = plt.figure(figsize=(14, 8))
    ax1 = fig.add_subplot(2, 1, 1);  ax2 = fig.add_subplot(2, 1, 2);  

    print(freq.shape, CAL_f_gain.shape)
    ax1.plot(freq, CAL_f_gain[0],  '-r', label='[U]', linewidth=1.6)
    ax1.plot(freq, CAL_f_gain[1],  '-g', label='[V]', linewidth=1.6)
    ax1.plot(freq, CAL_f_gain[2],  '-b', label='[W]', linewidth=1.6)
    ax2.plot(freq, CAL_f_phase[0], '-r', label='[U]', linewidth=1.6)
    ax2.plot(freq, CAL_f_phase[1], '-g', label='[V]', linewidth=1.6)
    ax2.plot(freq, CAL_f_phase[2], '-b', label='[W]', linewidth=1.6)

    ax1.set_xscale('log');  ax2.set_xscale('log');  ax1.set_yscale('log')
    ax1.set_ylabel('gain'); ax2.set_ylabel('phase')
    ax1.legend(loc='upper center', fontsize=8);  ax2.legend(loc='upper center', fontsize=8)
    xlim=[20, 45000]; ax1.set_xlim(xlim); ax2.set_xlim(xlim)
    fig.subplots_adjust(hspace=0)
    fig.show

    for i in range(3):
        for j in range(n_freq):
            if 1000 < freq[j]:
                break
        print(i, j, freq[j], CAL_f_gain[0][j])

In [ ]:
if cal_test>0:
    CAL_f_mode = 3

    freq = np.arange(5000)*10 + 20;  n_freq = freq.shape[0]
    T_HF  = 25;  T_RWI = [-175, -160, -140, -120, -100, -50, 25, 75]
    CAL_f_gain  = np.zeros((8, 3, n_freq))
    CAL_f_phase = np.zeros((8, 3, n_freq))
    for i in range(8):
        CAL_f_gain[i], CAL_f_phase[i] = juice_cal.spec_gain_phase(freq, CAL_f_mode, T_HF, T_RWI[i])

    fig = plt.figure(figsize=(14, 11))
    ax1 = fig.add_subplot(6, 1, 1);  ax2 = fig.add_subplot(6, 1, 2);  ax3 = fig.add_subplot(6, 1, 3)
    ax4 = fig.add_subplot(6, 1, 4);  ax5 = fig.add_subplot(6, 1, 5);  ax6 = fig.add_subplot(6, 1, 6)

    ax1.plot(freq, CAL_f_gain[0][0],  '--r', label='[U] RWI-T:'+format(T_RWI[0], '2'), linewidth=1.6)
    ax1.plot(freq, CAL_f_gain[1][0],  '--g', label='[U] RWI-T:'+format(T_RWI[1], '2'), linewidth=1.4)
    ax1.plot(freq, CAL_f_gain[2][0],  '--b', label='[U] RWI-T:'+format(T_RWI[2], '2'), linewidth=1.2)
    ax1.plot(freq, CAL_f_gain[3][0],  '-r',  label='[U] RWI-T:'+format(T_RWI[3], '2'), linewidth=1.0)
    ax1.plot(freq, CAL_f_gain[4][0],  '-g',  label='[U] RWI-T:'+format(T_RWI[4], '2'), linewidth=0.8)
    ax1.plot(freq, CAL_f_gain[5][0],  '-b',  label='[U] RWI-T:'+format(T_RWI[5], '2'), linewidth=0.6)
    ax1.plot(freq, CAL_f_gain[6][0],  '-r',  label='[U] RWI-T:'+format(T_RWI[6], '2'), linewidth=0.4)
    ax1.plot(freq, CAL_f_gain[7][0],  '-g',  label='[U] RWI-T:'+format(T_RWI[7], '2'), linewidth=0.2)
    ax2.plot(freq, CAL_f_gain[0][0],  '--r', label='[V] RWI-T:'+format(T_RWI[0], '2'), linewidth=1.6)
    ax2.plot(freq, CAL_f_gain[1][1],  '--g', label='[V] RWI-T:'+format(T_RWI[1], '2'), linewidth=1.4)
    ax2.plot(freq, CAL_f_gain[2][1],  '--b', label='[V] RWI-T:'+format(T_RWI[2], '2'), linewidth=1.2)
    ax2.plot(freq, CAL_f_gain[3][1],  '-r',  label='[V] RWI-T:'+format(T_RWI[3], '2'), linewidth=1.0)
    ax2.plot(freq, CAL_f_gain[4][1],  '-g',  label='[V] RWI-T:'+format(T_RWI[4], '2'), linewidth=0.8)
    ax2.plot(freq, CAL_f_gain[5][1],  '-b',  label='[V] RWI-T:'+format(T_RWI[5], '2'), linewidth=0.6)
    ax2.plot(freq, CAL_f_gain[6][1],  '-r',  label='[V] RWI-T:'+format(T_RWI[6], '2'), linewidth=0.4)
    ax2.plot(freq, CAL_f_gain[7][1],  '-g',  label='[V] RWI-T:'+format(T_RWI[7], '2'), linewidth=0.2)
    ax3.plot(freq, CAL_f_gain[0][2],  '--r', label='[W] RWI-T:'+format(T_RWI[0], '2'), linewidth=1.6)
    ax3.plot(freq, CAL_f_gain[1][2],  '--g', label='[W] RWI-T:'+format(T_RWI[1], '2'), linewidth=1.4)
    ax3.plot(freq, CAL_f_gain[2][2],  '--b', label='[W] RWI-T:'+format(T_RWI[2], '2'), linewidth=1.2)
    ax3.plot(freq, CAL_f_gain[3][2],  '-r',  label='[W] RWI-T:'+format(T_RWI[3], '2'), linewidth=1.0)
    ax3.plot(freq, CAL_f_gain[4][2],  '-g',  label='[W] RWI-T:'+format(T_RWI[4], '2'), linewidth=0.8)
    ax3.plot(freq, CAL_f_gain[5][2],  '-b',  label='[W] RWI-T:'+format(T_RWI[5], '2'), linewidth=0.6)
    ax3.plot(freq, CAL_f_gain[6][2],  '-r',  label='[W] RWI-T:'+format(T_RWI[6], '2'), linewidth=0.4)
    ax3.plot(freq, CAL_f_gain[7][2],  '-g',  label='[W] RWI-T:'+format(T_RWI[7], '2'), linewidth=0.2)

    ax4.plot(freq, CAL_f_phase[0][0], '--r', label='[U] RWI-T:'+format(T_RWI[0], '2'), linewidth=1.6)
    ax4.plot(freq, CAL_f_phase[1][0], '--g', label='[U] RWI-T:'+format(T_RWI[1], '2'), linewidth=1.4)
    ax4.plot(freq, CAL_f_phase[2][0], '--b', label='[U] RWI-T:'+format(T_RWI[2], '2'), linewidth=1.2)
    ax4.plot(freq, CAL_f_phase[3][0], '-r',  label='[U] RWI-T:'+format(T_RWI[3], '2'), linewidth=1.0)
    ax4.plot(freq, CAL_f_phase[4][0], '-g',  label='[U] RWI-T:'+format(T_RWI[4], '2'), linewidth=0.8)
    ax4.plot(freq, CAL_f_phase[5][0], '-b',  label='[U] RWI-T:'+format(T_RWI[5], '2'), linewidth=0.6)
    ax4.plot(freq, CAL_f_phase[6][0], '-r',  label='[U] RWI-T:'+format(T_RWI[6], '2'), linewidth=0.4)
    ax4.plot(freq, CAL_f_phase[7][0], '-g',  label='[U] RWI-T:'+format(T_RWI[7], '2'), linewidth=0.2)
    ax5.plot(freq, CAL_f_phase[0][1], '--r', label='[V] RWI-T:'+format(T_RWI[0], '2'), linewidth=1.6)
    ax5.plot(freq, CAL_f_phase[1][1], '--g', label='[V] RWI-T:'+format(T_RWI[1], '2'), linewidth=1.4)
    ax5.plot(freq, CAL_f_phase[2][1], '--b', label='[V] RWI-T:'+format(T_RWI[2], '2'), linewidth=1.2)
    ax5.plot(freq, CAL_f_phase[3][1], '-r',  label='[V] RWI-T:'+format(T_RWI[3], '2'), linewidth=1.0)
    ax5.plot(freq, CAL_f_phase[4][1], '-g',  label='[V] RWI-T:'+format(T_RWI[4], '2'), linewidth=0.8)
    ax5.plot(freq, CAL_f_phase[5][1], '-b',  label='[V] RWI-T:'+format(T_RWI[5], '2'), linewidth=0.6)
    ax5.plot(freq, CAL_f_phase[6][1], '-r',  label='[V] RWI-T:'+format(T_RWI[6], '2'), linewidth=0.4)
    ax5.plot(freq, CAL_f_phase[7][1], '-g',  label='[V] RWI-T:'+format(T_RWI[7], '2'), linewidth=0.2)
    ax6.plot(freq, CAL_f_phase[0][2], '--r', label='[W] RWI-T:'+format(T_RWI[0], '2'), linewidth=1.6)
    ax6.plot(freq, CAL_f_phase[1][2], '--g', label='[W] RWI-T:'+format(T_RWI[1], '2'), linewidth=1.4)
    ax6.plot(freq, CAL_f_phase[2][2], '--b', label='[W] RWI-T:'+format(T_RWI[2], '2'), linewidth=1.2)
    ax6.plot(freq, CAL_f_phase[3][2], '-r',  label='[W] RWI-T:'+format(T_RWI[3], '2'), linewidth=1.0)
    ax6.plot(freq, CAL_f_phase[4][2], '-g',  label='[W] RWI-T:'+format(T_RWI[4], '2'), linewidth=0.8)
    ax6.plot(freq, CAL_f_phase[5][2], '-b',  label='[W] RWI-T:'+format(T_RWI[5], '2'), linewidth=0.6)
    ax6.plot(freq, CAL_f_phase[6][2], '-r',  label='[W] RWI-T:'+format(T_RWI[6], '2'), linewidth=0.4)
    ax6.plot(freq, CAL_f_phase[7][2], '-g',  label='[W] RWI-T:'+format(T_RWI[7], '2'), linewidth=0.2)

    ax1.set_xscale('log');  ax2.set_xscale('log');  ax3.set_xscale('log');  ax4.set_xscale('log');  ax5.set_xscale('log');  ax6.set_xscale('log');    
    ax1.set_yscale('log');  ax2.set_yscale('log');  ax3.set_yscale('log');   
    ax1.set_ylabel('gain-U');  ax2.set_ylabel('gain-V');  ax3.set_ylabel('gain-W') 
    ax4.set_ylabel('phase-U'); ax5.set_ylabel('phase-V'); ax6.set_ylabel('phase-W')  
    ax1.legend(loc='upper center', fontsize=8);  ax2.legend(loc='upper center', fontsize=8);  ax3.legend(loc='upper center', fontsize=8)
    ax4.legend(loc='upper center', fontsize=8);  ax5.legend(loc='upper center', fontsize=8);  ax6.legend(loc='upper center', fontsize=8)
    xlim=[20, 45000]; ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim)

    fig.subplots_adjust(hspace=0)
    fig.show

In [ ]:
CAL_f_mode = 3

freq = np.arange(5000)*10 + 20;  n_freq = freq.shape[0]
T_HF  = [-30,   -15,    0,   15,   30,  45, 60, 75];  T_RWI = 25
CAL_f_gain  = np.zeros((8, 3, n_freq))
CAL_f_phase = np.zeros((8, 3, n_freq))
for i in range(8):
    CAL_f_gain[i], CAL_f_phase[i] = juice_cal.spec_gain_phase(freq, CAL_f_mode, T_HF[i], T_RWI)

fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(6, 1, 1);  ax2 = fig.add_subplot(6, 1, 2);  ax3 = fig.add_subplot(6, 1, 3)
ax4 = fig.add_subplot(6, 1, 4);  ax5 = fig.add_subplot(6, 1, 5);  ax6 = fig.add_subplot(6, 1, 6)

ax1.plot(freq, CAL_f_gain[0][0],  '--r', label='[U] HF-T:'+format(T_HF[0], '2'), linewidth=1.6)
ax1.plot(freq, CAL_f_gain[1][0],  '--g', label='[U] HF-T:'+format(T_HF[1], '2'), linewidth=1.4)
ax1.plot(freq, CAL_f_gain[2][0],  '--b', label='[U] HF-T:'+format(T_HF[2], '2'), linewidth=1.2)
ax1.plot(freq, CAL_f_gain[3][0],  '-r',  label='[U] HF-T:'+format(T_HF[3], '2'), linewidth=1.0)
ax1.plot(freq, CAL_f_gain[4][0],  '-g',  label='[U] HF-T:'+format(T_HF[4], '2'), linewidth=0.8)
ax1.plot(freq, CAL_f_gain[5][0],  '-b',  label='[U] HF-T:'+format(T_HF[5], '2'), linewidth=0.6)
ax1.plot(freq, CAL_f_gain[6][0],  '-r',  label='[U] HF-T:'+format(T_HF[6], '2'), linewidth=0.4)
ax1.plot(freq, CAL_f_gain[7][0],  '-g',  label='[U] HF-T:'+format(T_HF[7], '2'), linewidth=0.2)
ax2.plot(freq, CAL_f_gain[0][0],  '--r', label='[V] HF-T:'+format(T_HF[0], '2'), linewidth=1.6)
ax2.plot(freq, CAL_f_gain[1][1],  '--g', label='[V] HF-T:'+format(T_HF[1], '2'), linewidth=1.4)
ax2.plot(freq, CAL_f_gain[2][1],  '--b', label='[V] HF-T:'+format(T_HF[2], '2'), linewidth=1.2)
ax2.plot(freq, CAL_f_gain[3][1],  '-r',  label='[V] HF-T:'+format(T_HF[3], '2'), linewidth=1.0)
ax2.plot(freq, CAL_f_gain[4][1],  '-g',  label='[V] HF-T:'+format(T_HF[4], '2'), linewidth=0.8)
ax2.plot(freq, CAL_f_gain[5][1],  '-b',  label='[V] HF-T:'+format(T_HF[5], '2'), linewidth=0.6)
ax2.plot(freq, CAL_f_gain[6][1],  '-r',  label='[V] HF-T:'+format(T_HF[6], '2'), linewidth=0.4)
ax2.plot(freq, CAL_f_gain[7][1],  '-g',  label='[V] HF-T:'+format(T_HF[7], '2'), linewidth=0.2)
ax3.plot(freq, CAL_f_gain[0][2],  '--r', label='[W] HF-T:'+format(T_HF[0], '2'), linewidth=1.6)
ax3.plot(freq, CAL_f_gain[1][2],  '--g', label='[W] HF-T:'+format(T_HF[1], '2'), linewidth=1.4)
ax3.plot(freq, CAL_f_gain[2][2],  '--b', label='[W] HF-T:'+format(T_HF[2], '2'), linewidth=1.2)
ax3.plot(freq, CAL_f_gain[3][2],  '-r',  label='[W] HF-T:'+format(T_HF[3], '2'), linewidth=1.0)
ax3.plot(freq, CAL_f_gain[4][2],  '-g',  label='[W] HF-T:'+format(T_HF[4], '2'), linewidth=0.8)
ax3.plot(freq, CAL_f_gain[5][2],  '-b',  label='[W] HF-T:'+format(T_HF[5], '2'), linewidth=0.6)
ax3.plot(freq, CAL_f_gain[6][2],  '-r',  label='[W] HF-T:'+format(T_HF[6], '2'), linewidth=0.4)
ax3.plot(freq, CAL_f_gain[7][2],  '-g',  label='[W] HF-T:'+format(T_HF[7], '2'), linewidth=0.2)

ax4.plot(freq, CAL_f_phase[0][0], '--r', label='[U] HF-T:'+format(T_HF[0], '2'), linewidth=1.6)
ax4.plot(freq, CAL_f_phase[1][0], '--g', label='[U] HF-T:'+format(T_HF[1], '2'), linewidth=1.4)
ax4.plot(freq, CAL_f_phase[2][0], '--b', label='[U] HF-T:'+format(T_HF[2], '2'), linewidth=1.2)
ax4.plot(freq, CAL_f_phase[3][0], '-r',  label='[U] HF-T:'+format(T_HF[3], '2'), linewidth=1.0)
ax4.plot(freq, CAL_f_phase[4][0], '-g',  label='[U] HF-T:'+format(T_HF[4], '2'), linewidth=0.8)
ax4.plot(freq, CAL_f_phase[5][0], '-b',  label='[U] HF-T:'+format(T_HF[5], '2'), linewidth=0.6)
ax4.plot(freq, CAL_f_phase[6][0], '-r',  label='[U] HF-T:'+format(T_HF[6], '2'), linewidth=0.4)
ax4.plot(freq, CAL_f_phase[7][0], '-g',  label='[U] HF-T:'+format(T_HF[7], '2'), linewidth=0.2)
ax5.plot(freq, CAL_f_phase[0][1], '--r', label='[V] HF-T:'+format(T_HF[0], '2'), linewidth=1.6)
ax5.plot(freq, CAL_f_phase[1][1], '--g', label='[V] HF-T:'+format(T_HF[1], '2'), linewidth=1.4)
ax5.plot(freq, CAL_f_phase[2][1], '--b', label='[V] HF-T:'+format(T_HF[2], '2'), linewidth=1.2)
ax5.plot(freq, CAL_f_phase[3][1], '-r',  label='[V] HF-T:'+format(T_HF[3], '2'), linewidth=1.0)
ax5.plot(freq, CAL_f_phase[4][1], '-g',  label='[V] HF-T:'+format(T_HF[4], '2'), linewidth=0.8)
ax5.plot(freq, CAL_f_phase[5][1], '-b',  label='[V] HF-T:'+format(T_HF[5], '2'), linewidth=0.6)
ax5.plot(freq, CAL_f_phase[6][1], '-r',  label='[V] HF-T:'+format(T_HF[6], '2'), linewidth=0.4)
ax5.plot(freq, CAL_f_phase[7][1], '-g',  label='[V] HF-T:'+format(T_HF[7], '2'), linewidth=0.2)
ax6.plot(freq, CAL_f_phase[0][2], '--r', label='[W] HF-T:'+format(T_HF[0], '2'), linewidth=1.6)
ax6.plot(freq, CAL_f_phase[1][2], '--g', label='[W] HF-T:'+format(T_HF[1], '2'), linewidth=1.4)
ax6.plot(freq, CAL_f_phase[2][2], '--b', label='[W] HF-T:'+format(T_HF[2], '2'), linewidth=1.2)
ax6.plot(freq, CAL_f_phase[3][2], '-r',  label='[W] HF-T:'+format(T_HF[3], '2'), linewidth=1.0)
ax6.plot(freq, CAL_f_phase[4][2], '-g',  label='[W] HF-T:'+format(T_HF[4], '2'), linewidth=0.8)
ax6.plot(freq, CAL_f_phase[5][2], '-b',  label='[W] HF-T:'+format(T_HF[5], '2'), linewidth=0.6)
ax6.plot(freq, CAL_f_phase[6][2], '-r',  label='[W] HF-T:'+format(T_HF[6], '2'), linewidth=0.4)
ax6.plot(freq, CAL_f_phase[7][2], '-g',  label='[W] HF-T:'+format(T_HF[7], '2'), linewidth=0.2)

ax1.set_xscale('log');  ax2.set_xscale('log');  ax3.set_xscale('log');  ax4.set_xscale('log');  ax5.set_xscale('log');  ax6.set_xscale('log');    
ax1.set_yscale('log');  ax2.set_yscale('log');  ax3.set_yscale('log');   
ax1.set_ylabel('gain-U');  ax2.set_ylabel('gain-V');  ax3.set_ylabel('gain-W') 
ax4.set_ylabel('phase-U'); ax5.set_ylabel('phase-V'); ax6.set_ylabel('phase-W')  
ax1.legend(loc='upper center', fontsize=8);  ax2.legend(loc='upper center', fontsize=8);  ax3.legend(loc='upper center', fontsize=8)
ax4.legend(loc='upper center', fontsize=8);  ax5.legend(loc='upper center', fontsize=8);  ax6.legend(loc='upper center', fontsize=8)
xlim=[20, 45000]; ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim)

fig.subplots_adjust(hspace=0)
fig.show